In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
huggingface_dataset_name ="knkarthick/dialogsum"
dataset=load_dataset(huggingface_dataset_name)


Found cached dataset csv (/home/sagemaker-user/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
example_indices =[40,200]

dash_line='-'.join(''for x in range(100))
for i,index in enumerate (example_indices):
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()



---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exam

In [5]:
model_name ='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [6]:
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [8]:
sentence="What time is it,Tom?"

sentence_encoded=tokenizer(sentence, return_tensors='pt')

sentence_decoded=tokenizer.decode(
    sentence_encoded("input_ids")[0],
    skip_special_tokens=True
)
print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

TypeError: 'BatchEncoding' object is not callable

In [11]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    inputs=tokenizer(dialogue, return_tensors='pt')
    output=tokenizer.decode(model.generate(inputs["input_ids"],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION WITHOUT PROMPT ENGINEERING:\n{output}\n')

2024-01-27 02:55:40.371439: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

----------------------------------

In [15]:
for i,index in enumerate(example_indices):
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']
    prompt=f"""
Dialogue:

{dialogue}
 What was going on?
 """
    #Input constructed prompt instead of the dialogue
    inputs=tokenizer(prompt,return_tensors='pt')
    output=tokenizer.decode(model.generate(inputs["input_ids"],max_new_tokens=50,)[0],skip_special_tokens=True)
    print(dash_line)
    print('Example',i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION -ZERO SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
 What was going on?
 
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION -ZERO SHOT:
Tom is late for the train.
-----------------

#  ONE SHOT INFERENCE

In [18]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt=''
    for index in example_indices_full:
        dialogue=dataset['test'][index]['dialogue']
        summary=dataset['test'][index]['summary']
        prompt+=f"""
Dialogue:
{dialogue}

what was going on?
{summary}

"""
        dialogue=dataset['test'][example_index_to_summarize]['dialogue']
        prompt+=f"""
Dialogue:
{dialogue}

what was going on?
"""
        return prompt

In [19]:
example_indices_full=[40]
example_index_to_summarize=200

one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

what was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need 

In [20]:
summary=dataset['test'][example_index_to_summarize]['summary']

inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs["input_ids"],max_new_tokens=50,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION -ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION -ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [24]:
example_indices_full=[40,80,120]
example_index_to_summarize=200

few_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)

print(few_shot_prompt)


Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

what was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need 

In [26]:
summary=dataset['test'][example_index_to_summarize]['summary']

inputs=tokenizer(few_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs["input_ids"],max_new_tokens=50,)[0],skip_special_tokens=True)
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION -FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
MODEL GENERATION -FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.
